In [1]:
import os
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, ManagedIdentityCredential

In [2]:
AZURE_DIRECTORY = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/aimethica-general/code/Users/alexandru.popescu/quantity_sensitivity'

In [3]:
subscription_id = "9107822e-69ab-454f-a905-b59c4e15c1af"
resource_group = "AiMethica-ML"
workspace = "AiMethica-AZML"

In [4]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [5]:
ws = ml_client.workspaces.get(workspace)
print(ws.location, ":", ws.resource_group)

westeurope : AiMethica-ML


In [7]:
custom_env_name = 'aimethica-anomaly-env'
job_name = 'jolly_cat_f5vh1ymrw3'
env_name = custom_env_name + "@latest"

In [8]:
from azure.ai.ml.entities import Model

model = Model(
    # the script stores the model as "model"
    path=f"azureml://jobs/{job_name}/outputs/artifacts/paths/outputs/",
    name="quantity_sensitivity",
    description="Quantity sensitivity model, which predicts the company main KPIs based on different products quantities.",
    type="custom_model"
)

In [9]:
registered_model = ml_client.models.create_or_update(model=model)

In [10]:
online_endpoint_name = "quantity-sensitivity"

In [12]:
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Quantity sensitivity endpoint, which predicts the company main KPIs based on different products quantities.",
    auth_mode="key"
)

endpoint = ml_client.begin_create_or_update(endpoint).result()

print(f"Endpint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

Endpint quantity-sensitivity provisioning state: Succeeded


In [13]:
model_name = registered_model.name + '@latest'

code_directory = os.path.join(AZURE_DIRECTORY,'src')

blue_deployment = ManagedOnlineDeployment(
    name="tff-blue",
    endpoint_name=online_endpoint_name,
    model=model_name,
    code_configuration=CodeConfiguration(code=code_directory, scoring_script="score.py"),
    environment=env_name,
    instance_type="Standard_DS1_v2",
    instance_count=1,
)

In [14]:
blue_deployment = ml_client.begin_create_or_update(blue_deployment)

Instance type Standard_DS1_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/en-us/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint quantity-sensitivity exists


...................................................................................

In [15]:
anomaly_request = os.path.join(AZURE_DIRECTORY,"data/sensitivity_request.json")
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="tff-blue",
    request_file=anomaly_request
)

'{"response": [[647708.8125, 649525.25, 530646.25, 379830.125, 579610.8125]]}'